In [8]:
import pandas as pd
import random
import authorization
import numpy as np
from numpy.linalg import norm

In [9]:
df = pd.read_csv("valence_arousal_dataset.csv")
print(df.shape)
df.head()

(1234, 6)


,id,genre,track_name,artist_name,valence,energy
0,53rVYwGiKUC3gYX8SQQYYU,acoustic,Something In The Way,Nirvana,0.103,0.347
1,2FGb3gJbKToDII7q6Kfp4i,acoustic,I'm Yours,Ron Pope,0.866,0.553
2,2ZQyksYO4zzhyHNcueL0CP,acoustic,Home,Phillip Phillips,0.293,0.792
3,5GCgC77m6EaAqu7ZlukMu2,acoustic,Umbrella,Train,0.627,0.849
4,6vrUTGn5p8IrfTZ0J6sIVM,acoustic,Iris,The Goo Goo Dolls,0.513,0.790


In [10]:
df["mood_vec"] = df[["valence", "energy"]].values.tolist()
df["mood_vec"].head()

0    [0.103, 0.347]
1    [0.866, 0.553]
2    [0.293, 0.792]
3    [0.627, 0.849]
4     [0.513, 0.79]
Name: mood_vec, dtype: object

In [5]:
sp = authorization.authorize() 

In [6]:
def recommend(track_id, ref_df, sp, n_recs = 5):
    
    # Crawl valence and arousal of given track from spotify api
    track_features = sp.track_audio_features(track_id)
    track_moodvec = np.array([track_features.valence, track_features.energy])
    print(f"mood_vec for {track_id}: {track_moodvec}")
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
    # If the input track is in the reference set, it will have a distance of 0, but should not be recommendet
    ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    
    # Return n recommendations
    return ref_df_sorted.iloc[:n_recs]

In [11]:
track1 = random.choice(df["id"])
recommend(track_id = track1, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 49KmbdsI1k5LkdJwz64orV: [0.873 0.731]


,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
258,4wW9Xv0YkWydYIDDMRuy6X,detroit-techno,Elements Of Tone - J's A Mix,Various Artists,0.876,0.726,"[0.876, 0.726]",0.005831
564,4vWBetFgUQE4nZZgriHhwi,indian,Yeh Dil Na Hota Bechara,Kishore Kumar,0.876,0.723,"[0.876, 0.723]",0.008544
817,0HgiPF8K8ER71jHjdRNpNR,pagode,Menina Linda,Belo,0.873,0.722,"[0.873, 0.722]",0.009000
368,6LvuRWVuOMAhTPqxHsdV4Y,forro,A História do Peixe Tuninha,Pé de Mulambo,0.879,0.717,"[0.879, 0.717]",0.015232
385,7kQlRwniTCGhVny4V5whXZ,funk,Hot Pants... I'm Coming I'm Coming,Various Artists,0.886,0.739,"[0.886, 0.739]",0.015264
